In [1]:
import pandas as pd
from tensorflow.keras.experimental import LinearModel

2021-10-21 12:11:33.682161: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-21 12:11:33.682198: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import numpy as np

In [20]:
def softmax(x):
    return np.exp(x)/sum(np.exp(x))

In [4]:
train_set_path1 = "../data_sample/simple_ml_task/participants/participant_0/simple_ml_task_0.csv"
train_set_df1 = pd.read_csv(train_set_path1)
x_train1, y_train1 = train_set_df1[["x1", "x2"]].values[:10], train_set_df1["y"].values[:10]

In [5]:
train_set_path2 = "../data_sample/simple_ml_task/participants/participant_1/simple_ml_task_1.csv"
train_set_df2 = pd.read_csv(train_set_path2)
x_train2, y_train2 = train_set_df2[["x1", "x2"]].values, train_set_df2["y"].values

In [6]:
model1 = LinearModel(activation="sigmoid")
model1.compile(optimizer="sgd", loss="mse", metrics="accuracy")

model2 = LinearModel(activation="sigmoid")
model2.compile(optimizer="sgd", loss="mse", metrics="accuracy")

2021-10-21 12:11:35.622620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-21 12:11:35.622653: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-21 12:11:35.622678: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Lenovo-ideapad): /proc/driver/nvidia/version does not exist
2021-10-21 12:11:35.623040: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model1.fit(x_train1, y_train1, epochs=3)
model2.fit(x_train2, y_train2, epochs=3)

2021-10-21 12:11:35.722222: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3
1/1 [==============================] - 0s 377ms/step - loss: 0.2500 - accuracy: 0.7000
Epoch 2/3
1/1 [==============================] - 0s 3ms/step - loss: 0.2224 - accuracy: 0.7000
Epoch 3/3
1/1 [==============================] - 0s 4ms/step - loss: 0.2063 - accuracy: 0.7000
Epoch 1/3
7/7 [==============================] - 0s 1ms/step - loss: 0.2293 - accuracy: 0.8350
Epoch 2/3
7/7 [==============================] - 0s 2ms/step - loss: 0.1824 - accuracy: 1.0000
Epoch 3/3
7/7 [==============================] - 0s 1ms/step - loss: 0.1467 - accuracy: 1.0000


In [8]:
models = [model1, model2]

In [9]:
test_set_path = "../data_sample/simple_ml_task/validator/simple_ml_task_test.csv"
test_set_df = pd.read_csv(test_set_path)
x_test, y_test = test_set_df[["x1", "x2"]].values, test_set_df["y"].values

In [10]:
score1 = model1.evaluate(x_test, y_test)[1]
score2 = model2.evaluate(x_test, y_test)[1]

4/4 [==============================] - 0s 1ms/step - loss: 0.1280 - accuracy: 1.0000


In [11]:
scores = [score1, score2]

In [22]:
from scipy.special import softmax as sc_softmax

In [21]:
alpha = softmax(scores)
print(alpha)

[0.36586441 0.63413559]


In [23]:
alpha = sc_softmax(scores)
print(alpha)

[0.36586441 0.63413559]


In [13]:
global_model = LinearModel(activation="sigmoid")
global_model.compile(optimizer="sgd", loss="mse", metrics="accuracy")
global_model.fit(x_train1, y_train1)

1/1 [==============================] - 0s 306ms/step - loss: 0.2500 - accuracy: 0.7000


In [14]:
weights_global_model = model1.get_weights()
new_weights = []
for layer_weights in weights_global_model:
    new_weights.append(np.zeros(layer_weights.shape))
    
for a_i, model in zip(alpha, models):
    weights = model.get_weights()
    for idx_layer, layer_weights in enumerate(weights):
        new_weights[idx_layer] = new_weights[idx_layer] + layer_weights * a_i 
global_model.set_weights(new_weights)

In [15]:
global_model.evaluate(x_test, y_test)

4/4 [==============================] - 0s 1ms/step - loss: 0.1670 - accuracy: 0.9600


[0.16698062419891357, 0.9599999785423279]